In [1]:
# Collect spec data into one file

import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

# Paths
root = '/media/latlab/MR/projects/kaggle-hms'
data_dir = os.path.join(root, 'data')
results_dir = os.path.join(root, 'results')
train_eeg_dir = os.path.join(data_dir, 'train_eegs')
train_spec_dir = os.path.join(data_dir, 'train_spectrograms')

# Load data
df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
df

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,351917269,6,12.0,2147388374,6,12.0,4195677307,10351,LRDA,0,0,0,3,0,0
106796,351917269,7,14.0,2147388374,7,14.0,290896675,10351,LRDA,0,0,0,3,0,0
106797,351917269,8,16.0,2147388374,8,16.0,461435451,10351,LRDA,0,0,0,3,0,0
106798,351917269,9,18.0,2147388374,9,18.0,3786213131,10351,LRDA,0,0,0,3,0,0


In [2]:
%%time
time_str_list = [
    '_0.59', '_0.78', '_0.98', '_1.17', '_1.37', '_1.56', '_1.76', '_1.95', '_2.15', '_2.34', '_2.54', '_2.73', '_2.93', '_3.13', '_3.32', 
    '_3.52', '_3.71', '_3.91', '_4.1', '_4.3', '_4.49', '_4.69', '_4.88', '_5.08', '_5.27', '_5.47', '_5.66', '_5.86', '_6.05', '_6.25', 
    '_6.45', '_6.64', '_6.84', '_7.03', '_7.23', '_7.42', '_7.62', '_7.81', '_8.01', '_8.2', '_8.4', '_8.59', '_8.79', '_8.98', '_9.18', 
    '_9.38', '_9.57', '_9.77', '_9.96', '_10.16', '_10.35', '_10.55', '_10.74', '_10.94', '_11.13', '_11.33', '_11.52', '_11.72', '_11.91', 
    '_12.11', '_12.3', '_12.5', '_12.7', '_12.89', '_13.09', '_13.28', '_13.48', '_13.67', '_13.87', '_14.06', '_14.26', '_14.45', '_14.65', 
    '_14.84', '_15.04', '_15.23', '_15.43', '_15.63', '_15.82', '_16.02', '_16.21', '_16.41', '_16.6', '_16.8', '_16.99', '_17.19', '_17.38', 
    '_17.58', '_17.77', '_17.97', '_18.16', '_18.36', '_18.55', '_18.75', '_18.95', '_19.14', '_19.34', '_19.53', '_19.73', '_19.92']
col_str_list = ['LL' + x for x in time_str_list] + ['RL' + x for x in time_str_list] + ['LP' + x for x in time_str_list] + ['RP' + x for x in time_str_list]

spec_data = dict()
for spec_id in tqdm(df['spectrogram_id'].unique()):
    spec_path = os.path.join(train_spec_dir, f'{spec_id}.parquet')
    curr_spec_data = pd.read_parquet(spec_path)

    assert curr_spec_data['time'][0] == 1
    assert (curr_spec_data['time'].diff()[1:] == 2.0).all()
    assert (curr_spec_data.columns[1:] == col_str_list).all()
    
    curr_spec_data = curr_spec_data.drop(columns=['time']).to_numpy()
    spec_data[spec_id] = curr_spec_data


100%|██████████| 11138/11138 [07:07<00:00, 26.03it/s]

CPU times: user 13min 25s, sys: 3min 3s, total: 16min 29s
Wall time: 7min 7s


In [3]:
np.save(os.path.join(data_dir, 'spec_data.npy'), spec_data)

In [4]:
%%time
test_data = np.load(os.path.join(data_dir, 'spec_data.npy'), allow_pickle=True).item()

CPU times: user 0 ns, sys: 3.29 s, total: 3.29 s
Wall time: 4.39 s
